In [1]:
import numpy as np
from qmrdmft import Calculator, OptimalCalculator, tools
from pyscf import gto, scf, ao2mo
from numba import prange, jit, njit

#import tools

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE


$$
\begin{split}
E_H[\gamma_1] & =  \sum_a^M \sum_b^M n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{a,\kappa} \tilde{\tilde{c}}_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
E_H[\gamma_1] & =  \iint \sum_a^M \sum_b^M n_a n_b \frac{\eta_a(r)^{\ast} \eta_a(r) \eta_b(r')^{\ast}  \eta_b(r')}{|r-r'|} dr dr' \\
E_H[\gamma_1] & =  \iint \frac{\gamma_1(r,r) \gamma_1(r',r')}{|r-r'|} dr dr' \\
\end{split}
$$

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

The Geodecker Umrigar Hartree-like Term

$$
\begin{split}
^{GU}E_H[\gamma_1] & =  \sum_a^M' \sum_b^M' n_a n_b \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{a,\kappa} \tilde{\tilde{c}}_{b,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
^{GU}E_H[\gamma_1] & =  \iint \sum_a^M' \sum_b^M' n_a n_b \frac{\eta_a(r)^{\ast} \eta_a(r) \eta_b(r')^{\ast}  \eta_b(r')}{|r-r'|} dr dr' \\
^{GU}E_H[\gamma_1] & =  \iint \frac{\gamma_1'(r,r) \gamma_1'(r',r')}{|r-r'|} dr dr' \\
\end{split}
$$
Here the prime indicates that terms in which $a=b$ are .left out of the sum, and the out of the sume consituting $\gamma_1$.

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [4]:
# for spinorbitals, the Umrigar Hartree term is a Hartree term from which the self interaction has been removed 
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

The Geodecker Umrigar Exchange-like Term, since the self interaction compensation is altready done in the Hartree like term, it is inaccurate to call this term exchange term. 


$$
\begin{split}
^{GU}E_{xc}[\gamma_1] & = -\frac{1}{2} \sum_a^M' \sum_b^M' \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{b,\kappa} \tilde{\tilde{c}}_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
^{GU}E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \sum_a^M' \sum_b^M' \sqrt{n_a n_b} \frac{\eta_a(r)^{\ast} \eta_b(r) \eta_b(r')^{\ast}  \eta_a(r')}{|r-r'|} dr dr' \\
^{GU}E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \frac{\gamma_1'^{\frac{1}{2}}(r,r') \gamma_1'^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

Here the prime indicates that terms in which $a=b$ are .left out of the sum, and the out of the sume consituting $\gamma_1$.

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [5]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy


$$
\begin{split}
^{MU}E_{xc}[\gamma_1] & = -\frac{1}{2} \sum_a^M \sum_b^M \sqrt{n_a n_b} \sum_{\mu}^M \sum_{\nu}^M \sum_{\kappa}^M \sum_{\lambda}^M  \tilde{\tilde{c}}_{a,\mu} \tilde{\tilde{c}}_{b,\nu} \tilde{\tilde{c}}_{b,\kappa} \tilde{\tilde{c}}_{a,\lambda} \left [ \mu \nu | \kappa \lambda \right ] \\
^{MU}E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \sum_a^M \sum_b^M \sqrt{n_a n_b} \frac{\eta_a(r)^{\ast} \eta_b(r) \eta_b(r')^{\ast}  \eta_a(r')}{|r-r'|} dr dr' \\
^{MU}E_{xc}[\gamma_1] & = -\frac{1}{2} \iint \frac{\gamma_1^{\frac{1}{2}}(r,r') \gamma_1^{\frac{1}{2}}(r',r)}{|r-r'|} dr dr' \\
\end{split}
$$

where $\{\eta_{a}\}$ is the set of natural orbitals, $M$ the number of basis functions $\{\chi_{\mu}\}$ and of natural orbitals. The coefficients $\tilde{\tilde{c}}$ represent $\eta$ in the basis $\chi$.

In [6]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  range(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  range(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [7]:
element = "Li"
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0; {element} 0 0 1'''
mol.charge = 0
mol.spin = 0
mol.basis = 'Sto-3g'
#mol.basis = "6-31g" 
mol.unit = 'AU'
mol.build()

In [8]:
# get molecuar quantities 
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')
N_a, N_b = mol.nelec
M = eri.shape[0]

In [9]:
hf = scf.HF(mol).run()

converged SCF energy = -12.9518686845497


In [10]:
# get fock properties
C = hf.mo_coeff
h = hf.get_hcore()
J = hf.get_j()
K = hf.get_k()

#print(K.shape)
#print(J.shape)

In [11]:
# AO MO transformation of elements of fock operator
h1 = C.T@h@C

#JKmo = C.T@(J+K)@C
h2 = ao2mo.kernel(mol, hf.mo_coeff)
#print(JKmo)

In [12]:
e, rdms1 = tools.e_rdms(h1, h2, mol.energy_nuc(), mol.nelec, h1.shape[1],  nroots=3)

In [13]:
V_ee = e[0]-np.trace(np.matmul(h1,rdms1[0]))

In [14]:
## LFT of Levy Valone

eq_cons = {'type': 'eq',
           'fun' : lambda x: x[0]-h1[0, 0]}
# initializing the calculator
calculator = Calculator(h2, rdms1[0].copy(), # two body term and rdm for the ground state
                        tools.e_rdms, tools.optimize, 1e-7, # function that calculates (energies, rdms) and optimizer 
                        ecore=mol.energy_nuc(), norbs=h1.shape[1], nelec=mol.nelec, maximize=True, symmetrize=True, nroots=5) # parameters for the

# get the exact value of the objective function
L, dL = calculator.calculate_objctv_grdnt(h1)
#print("Exact value of the objective function", L)

# run the optimizer
## objectiva function value changes when we add constraint
res = calculator.optimize(np.zeros(h1.size), 
                          method='trust-constr',
                          options={"maxiter":500, 'disp':False, 'verbose':0, 'gtol':1e-7,},
                          constraints=eq_cons)

print("Print value after optimization: ", res.fun, res.fun+V_ee) 
W=res.fun
#print("Difference:", res.fun-L)

/home/jerhard/1RDMFT/lib/python3.12/site-packages/scipy/optimize/_differentiable_functions.py:551: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(delta_x, delta_g)
/home/jerhard/1RDMFT/QMRDMFT/qmrdmft/calculator.py:119: SyntaxWarning: invalid escape sequence '\m'
  """Calculator class
/home/jerhard/1RDMFT/QMRDMFT/qmrdmft/calculator.py:196: SyntaxWarning: invalid escape sequence '\m'
  """OptimalCalculator class


KeyboardInterrupt: 

In [ ]:
FCIoccu, FCInaturalC = np.linalg.eigh(rdms1[0]/2.0)
#naturalCTT = np.matmul(C,naturalC) # I think this might just be a diagonal matrix  
FCInaturalCTT = np.matmul(C,FCInaturalC)
FCInaturalCTTE = expand_matrix(FCInaturalCTT)
FCIoccuE = np.append(FCIoccu,FCIoccu)

print(FCIoccuE,  sum(FCIoccuE))

In [ ]:
Um_2 = ONERDMFT_Umrigar_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE) # these routines expect all the maytrices to be expanded first
Um_1 = ONERDMFT_Umrigar_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
H    = ONERDMFT_hartree_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)
Mu   = ONERDMFT_Mueller_exchange_correlation_energy_parallel(eri, FCInaturalCTTE, FCIoccuE)

print(f"{Um_1:2.4f} {Um_2:2.4f} {H:2.4f} {Mu:2.4f}")

In [ ]:
print(.5*np.trace(np.matmul(rdms1[0],J)))

In [ ]:
print(f"{2*(Um_1+Um_2)}  {2*(H+Mu)}  {V_ee}")